In [ ]:
#### HSV approach total contour detection Experiments ######

import cv2
import numpy as np

img = cv2.imread('./defect_2/DJI_0173_mod3.JPG')
rows = img.shape[0]
cols = img.shape[1]
rows = int((rows*40)/100)
cols = int((cols*40)/100)
img = cv2.resize(img, (cols,rows))
mask_contour = np.zeros((rows,cols), np.uint8)
img_out1 = img.copy()
img_out2 =img.copy()

hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
red = hsv[:,:,0]
rettt,thresh7 = cv2.threshold(red,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

cv2.imshow('shadows_out11', thresh7)
remove_noise = cv2.medianBlur(thresh7,3)
remove_noise2 = cv2.medianBlur(remove_noise,3)
remove_noise3 = cv2.medianBlur(remove_noise2,5)
# edges7 = cv2.Canny(remove_noise,0,255)
contours7, _ = cv2.findContours(thresh7,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
c = max(contours7, key = cv2.contourArea)
# print(c)
result = cv2.drawContours(mask_contour, [c], 0, (255, 255, 255), -1, cv2.LINE_AA)
kernel = np.ones((3,3),np.uint8)
dilation = cv2.dilate(result,kernel,iterations = 5)
filled = dilation

cv2.imshow("Result",dilation)
cv2.waitKey(0)
cv2.destroyAllWindows()



result_cord= []
coord = []
for i1 in range(rows-1):
    for j1 in range(cols-1):
        pixel1 = filled[i1,j1]
        pixel2 =  filled[i1,j1+1]
        result = abs(pixel2 - pixel1)
        if result>0:
            coord = (j1+1,i1)
#             print(coord)
            result_cord.append(coord)
# print('previous',result_cord)      
row_white = []
col_white = []
whitecoordi = []
whitecoordi2 = []
for i11 in range(rows):
     for j11 in range(cols):
        white = filled[i11,j11]
        if (i11==0 and white>0) or (i11==rows-1 and white>0):
            whitecoordi = (j11,i11)
            row_white.append(whitecoordi)
        if (j11==0 and white>0) or (j11==cols-1 and white>0):
            whitecoordi2 = (j11,i11)
            col_white.append(whitecoordi2)


white_coordi = row_white + col_white + result_cord
white_coordi = np.array(white_coordi)

def last(n):  
    return n[m]   
def sort(tuples): 
    return sorted(tuples, key = last) 
m = 0

### Convex hull - outer contour ###

hull = cv2.convexHull(white_coordi)
hull_coord1 = []
hull_coord2 = []
for i4 in range(len(hull)):
    hull_coord = hull[i4]
    x4, y4 = hull_coord[0]
    hull_coord1 = (x4, y4)
    hull_coord2.append(hull_coord1)
hull_new =  row_white + col_white + hull_coord2 
hull_new = np.array(hull_new)
hull_sorted_pts = sort(hull_new)
hull_update_pts = hull_sorted_pts
def euclidean1(coords):
    x0, y0 = hull_update_pts[0]
    x, y = coords
    return ((x-x0)**2 + (y-y0)**2)**0.5
hull_first_ele = []
for i2 in range(len(hull_update_pts)-1):
    candidates = hull_update_pts[1:]
#     print('SORTED\n',candidates)
    candidates.sort(key=euclidean1)
    hull_update_pts = candidates  
    hull_first_ele.append(candidates[0])

hull_first_ele = np.array(hull_first_ele[1:])

def euclidean2(coords):
    x60, y60 = hull_first_ele[-1]
    x6, y6 = coords
    return ((x6-x60)**2 + (y6-y60)**2)**0.5
connect = sort(hull_first_ele[:100])
for i66 in range(len(connect)):
    connect.sort(key = euclidean2)
f0,f1 = connect[0] 
f2,f3 = hull_first_ele[-1]
# for i55 in range(len(hull_first_ele)-1):
#     x50,y50 = hull_first_ele[i55]
#     x51,y51 = hull_first_ele[i55+1]
#     lineThickness = 2
#     cv2.line(img, (x50,y50), (x51,y51), (0,0,255), lineThickness)
#     cv2.line(img, (f0,f1), (f2,f3), (0,0,255), lineThickness)
hull_first_ele2 =[]
hull_first_ele3 =[]
for i70 in range(len(hull_first_ele)):
    f2,f3 = hull_first_ele[i70]
    if (f2,f3)!=(f0,f1):
        hull_first_ele2 = (f2,f3)
        hull_first_ele3.append(hull_first_ele2)
    
hull_first_ele3 = [(f0,f1)]+hull_first_ele3
hull_first_ele3 = np.array(hull_first_ele3)
cv2.drawContours(img_out1, [hull_first_ele3],  -1, (0, 0, 255), 2, cv2.LINE_AA)

mask_normal = np.zeros((rows,cols), np.uint8)
cv2.drawContours(mask_normal, [hull_first_ele3], 0, (255, 255, 255), -1, cv2.LINE_AA)
# mask1 = cv2.bitwise_and(img, img, mask=mask)


#### CONVEX HULL ENDS ###


######### Area contour begins#######

sorted_pts = sort(white_coordi)

update_pts = sorted_pts
def euclidean(coords):
    x01, y01 = update_pts[0]
    x011, y011 = coords
    return ((x011-x01)**2 + (y011-y01)**2)**0.5
first_ele = []
for i2 in range(len(update_pts)-1):
    candidates = update_pts[1:]
    candidates.sort(key=euclidean)
    update_pts = candidates
#     print('SORTED\n',candidates)
    first_ele.append(candidates[0])

# first_ele.insert(0,sorted_pts[0])
# print('\n NEW\n',first_ele)

# print('\nsorted\n',first_ele) 
values = []
central_coordi = []
#### Angle ####
for i5 in range(len(first_ele)-2):
    x5,y5 = first_ele[i5]
    x15,y15 = first_ele[i5+1]
    x25,y25 = first_ele[i5+2]
    angle2 = np.arctan((y25 - y15)/(x25 - x15 +0.00005)) - np.arctan((y15-y5)/(x15-x5 +0.00005))
    angle2 = np.degrees(angle2)
    if angle2 < 0:
        angle2 = angle2 + 180.0
    values.append(angle2)
    central_coordi.append(first_ele[i5+1])
#     print('Angle',angle2)
#     print('Coordinate',first_ele[i5+1])
  
img1 = img.copy()
img2 = img.copy()
MM=[]
angle_val = []
for i6 in range(len(values)-1):
    val1 = values[i6]
    val2 = values[i6+1]
    val3 = abs(val2-val1)
    X1,Y1 = central_coordi[i6]
    X2,Y2 = central_coordi[i6+1]
    val4 = abs(X2-X1)
    M = (val3+0.00005)/(val4+0.00005)
    MM.append(M)
    angle_val.append(val3)
#     print('Measure',M)
#     print('Coordinate',central_coordi[i6])

medi1 = np.mean(MM)
# print('Mean',medi1)
coordd = []
modfies_coord = []
for i9 in range(len(MM)):
    if MM[i9]>medi1*4.5:
            X1,Y1 = central_coordi[i9]
            coordd = (X1,Y1)
            modfies_coord.append(coordd)
#             cv2.circle(img1,(X1,Y1),2,255,-1)
coordd2 = []
modfied_coord2 = []
for i10 in range(len(modfies_coord)-1):
    x10,y10 = modfies_coord[i10]
    x100,y100 = modfies_coord[i10+1]

    distance = ((x100-x10)**2 + (y100-y10)**2)**0.5
    if distance > 6:
#         print('Distance',distance)
#         cv2.circle(img1,(x10,y10 ),2,255,-1)
        coordd2 = (x10,y10)
        modfied_coord2.append(coordd2)
coordd3 = []
modfied_coord3 = []
for i20 in range(len(modfied_coord2)-2):
    x20,y20 = modfied_coord2[i20]
    x21,y21 = modfied_coord2[i20+1]
    x22,y22 = modfied_coord2[i20+2]
    angle21 = np.arctan((y22 - y21)/(x22 - x21 +0.00005)) - np.arctan((y21-y20)/(x21-x20 +0.00005))
    angle21 = np.degrees(angle21)
    if angle21 < 0:
        angle21 = angle21 + 180.0
    if angle21>70:
        cv2.circle(img2,(x21,y21),2,255,-1)
#         print('Angle_final',angle21)
        coordd3 = (x21,y21)
        modfied_coord3.append(coordd3)
area_coord = []
area_coord2 = []
for i30 in range(len(modfied_coord3)-2):
    x30,y30 = modfied_coord3[i30]
    x31,y31 = modfied_coord3[i30+1]
    x32,y32 = modfied_coord3[i30+2]
    area = abs((x30*(y31-y32)+x31*(y32-y30)+x32*(y30-y31))/2)
#     print('Area',area)
    if area > 50:
        cv2.circle(img_out2,(x31,y31),2,255,-1)
        area_coord = (x31,y31)
        area_coord2.append(area_coord)
#         print('coord',(x31,y31))

area_coord3 = row_white + col_white + area_coord2
area_coord3 =np.array(area_coord3)

area_sorted_pts = sort(area_coord3)

area_update_pts = area_sorted_pts
def euclidean4(coords):
    x04, y04 = area_update_pts[0]
    x044, y044 = coords
    return ((x044-x04)**2 + (y044-y04)**2)**0.5
area_first_ele = []
for i80 in range(len(area_update_pts)-1):
    area_candidates = area_update_pts[1:]
#     print('SORTED\n',candidates)
    area_candidates.sort(key=euclidean4)
    area_update_pts = area_candidates  
    area_first_ele.append(area_candidates[0])

area_first_ele = np.array(area_first_ele[1:])
# print(area_first_ele)
cv2.drawContours(img_out2, [area_first_ele],  -1, (0, 0, 255), 2, cv2.LINE_AA)

mask_defect = np.zeros((rows,cols), np.uint8)
cv2.drawContours(mask_defect, [area_first_ele], 0, (255, 255, 255), -1, cv2.LINE_AA)
mask_result = cv2.bitwise_xor(mask_normal,mask_defect)


cv2.imshow("Normal", mask_normal)
# cv2.imwrite('Normal_contour.jpg',mask_normal)
cv2.imshow("Defected", mask_defect)
# cv2.imwrite('Defected_contour.jpg',mask_defect)
cv2.imshow("rrr", mask_result)
# cv2.imwrite('Defected_Part.jpg',mask_result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
 ### combined- normal + defect contour ###
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
img = cv2.imread('./defect_2/DJI_0173_mod3.JPG')
rows = img.shape[0]
cols = img.shape[1]
rows = int((rows*40)/100)
cols = int((cols*40)/100)
img = cv2.resize(img, (cols,rows))
mask_contour = np.zeros((rows,cols), np.uint8)
img_out1 = img.copy()
img_out2 =img.copy()

hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
red = hsv[:,:,0]
rettt,thresh7 = cv2.threshold(red,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# cv2.imshow('shadows_out11', thresh7)
remove_noise = cv2.medianBlur(thresh7,3)
remove_noise2 = cv2.medianBlur(remove_noise,3)
remove_noise3 = cv2.medianBlur(remove_noise2,5)
# edges7 = cv2.Canny(remove_noise,0,255)
contours7, _ = cv2.findContours(thresh7,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
c = max(contours7, key = cv2.contourArea)
# print(c)
result = cv2.drawContours(mask_contour, [c], 0, (255, 255, 255), -1, cv2.LINE_AA)
kernel = np.ones((3,3),np.uint8)
dilation = cv2.dilate(result,kernel,iterations = 5)
filled = dilation


# filled = ~filled

result_cord= []
coord = []
for i1 in range(rows-1):
    for j1 in range(cols-1):
        pixel1 = filled[i1,j1]
        pixel2 =  filled[i1,j1+1]
        result = abs(pixel2 - pixel1)
        if result>0:
            coord = (j1+1,i1)
#             print(coord)
            result_cord.append(coord)
# print('previous',result_cord)      
row_white = []
col_white = []
whitecoordi = []
whitecoordi2 = []
for i11 in range(rows):
     for j11 in range(cols):
        white = filled[i11,j11]
        if (i11==0 and white>0) or (i11==rows-1 and white>0):
            whitecoordi = (j11,i11)
            row_white.append(whitecoordi)
        if (j11==0 and white>0) or (j11==cols-1 and white>0):
            whitecoordi2 = (j11,i11)
            col_white.append(whitecoordi2)


white_coordi = row_white + col_white + result_cord
white_coordi = np.array(white_coordi)

def last(n):  
    return n[m]   
def sort(tuples): 
    return sorted(tuples, key = last) 
m = 0

### Convex hull - outer contour ###

hull = cv2.convexHull(white_coordi)
hull_coord1 = []
hull_coord2 = []
for i4 in range(len(hull)):
    hull_coord = hull[i4]
    x4, y4 = hull_coord[0]
    hull_coord1 = (x4, y4)
    hull_coord2.append(hull_coord1)
hull_new =  row_white + col_white + hull_coord2 
hull_new = np.array(hull_new)
hull_sorted_pts = sort(hull_new)
hull_update_pts = hull_sorted_pts
def euclidean1(coords):
    x0, y0 = hull_update_pts[0]
    x, y = coords
    return ((x-x0)**2 + (y-y0)**2)**0.5
hull_first_ele = []
for i2 in range(len(hull_update_pts)-1):
    candidates = hull_update_pts[1:]
#     print('SORTED\n',candidates)
    candidates.sort(key=euclidean1)
    hull_update_pts = candidates  
    hull_first_ele.append(candidates[0])

hull_first_ele = np.array(hull_first_ele[1:])

def euclidean2(coords):
    x60, y60 = hull_first_ele[-1]
    x6, y6 = coords
    return ((x6-x60)**2 + (y6-y60)**2)**0.5
connect = sort(hull_first_ele[:100])
for i66 in range(len(connect)):
    connect.sort(key = euclidean2)
f0,f1 = connect[0] 
f2,f3 = hull_first_ele[-1]
# for i55 in range(len(hull_first_ele)-1):
#     x50,y50 = hull_first_ele[i55]
#     x51,y51 = hull_first_ele[i55+1]
#     lineThickness = 2
#     cv2.line(img, (x50,y50), (x51,y51), (0,0,255), lineThickness)
#     cv2.line(img, (f0,f1), (f2,f3), (0,0,255), lineThickness)
hull_first_ele2 =[]
hull_first_ele3 =[]
for i70 in range(len(hull_first_ele)):
    f2,f3 = hull_first_ele[i70]
    if (f2,f3)!=(f0,f1):
        hull_first_ele2 = (f2,f3)
        hull_first_ele3.append(hull_first_ele2)
    
hull_first_ele3 = [(f0,f1)]+hull_first_ele3
hull_first_ele3 = np.array(hull_first_ele3)
cv2.drawContours(img_out1, [hull_first_ele3],  -1, (0, 0, 255), 2, cv2.LINE_AA)

mask_normal = np.zeros((rows,cols), np.uint8)
cv2.drawContours(mask_normal, [hull_first_ele3], 0, (255, 255, 255), -1, cv2.LINE_AA)
# mask1 = cv2.bitwise_and(img, img, mask=mask)


#### CONVEX HULL ENDS ###


######### Area contour begins#######

sorted_pts = sort(white_coordi)

update_pts = sorted_pts
def euclidean(coords):
    x01, y01 = update_pts[0]
    x011, y011 = coords
    return ((x011-x01)**2 + (y011-y01)**2)**0.5
first_ele = []
for i2 in range(len(update_pts)-1):
    candidates = update_pts[1:]
    candidates.sort(key=euclidean)
    update_pts = candidates
#     print('SORTED\n',candidates)
    first_ele.append(candidates[0])

# first_ele.insert(0,sorted_pts[0])
# print('\n NEW\n',first_ele)

# print('\nsorted\n',first_ele) 
values = []
central_coordi = []
#### Angle ####
for i5 in range(len(first_ele)-2):
    x5,y5 = first_ele[i5]
    x15,y15 = first_ele[i5+1]
    x25,y25 = first_ele[i5+2]
    angle2 = np.arctan((y25 - y15)/(x25 - x15 +0.00005)) - np.arctan((y15-y5)/(x15-x5 +0.00005))
    angle2 = np.degrees(angle2)
    if angle2 < 0:
        angle2 = angle2 + 180.0
    values.append(angle2)
    central_coordi.append(first_ele[i5+1])
#     print('Angle',angle2)
#     print('Coordinate',first_ele[i5+1])
  
img1 = img.copy()
img2 = img.copy()
MM=[]
angle_val = []
for i6 in range(len(values)-1):
    val1 = values[i6]
    val2 = values[i6+1]
    val3 = abs(val2-val1)
    X1,Y1 = central_coordi[i6]
    X2,Y2 = central_coordi[i6+1]
    val4 = abs(X2-X1)
    M = (val3+0.00005)/(val4+0.00005)
    MM.append(M)
    angle_val.append(val3)
#     print('Measure',M)
#     print('Coordinate',central_coordi[i6])

medi1 = np.mean(MM)
# print('Mean',medi1)
coordd = []
modfies_coord = []
for i9 in range(len(MM)):
    if MM[i9]>medi1*4.5:
            X1,Y1 = central_coordi[i9]
            coordd = (X1,Y1)
            modfies_coord.append(coordd)
#             cv2.circle(img1,(X1,Y1),2,255,-1)
coordd2 = []
modfied_coord2 = []
for i10 in range(len(modfies_coord)-1):
    x10,y10 = modfies_coord[i10]
    x100,y100 = modfies_coord[i10+1]

    distance = ((x100-x10)**2 + (y100-y10)**2)**0.5
    if distance > 6:
#         print('Distance',distance)
#         cv2.circle(img1,(x10,y10 ),2,255,-1)
        coordd2 = (x10,y10)
        modfied_coord2.append(coordd2)
coordd3 = []
modfied_coord3 = []
for i20 in range(len(modfied_coord2)-2):
    x20,y20 = modfied_coord2[i20]
    x21,y21 = modfied_coord2[i20+1]
    x22,y22 = modfied_coord2[i20+2]
    angle21 = np.arctan((y22 - y21)/(x22 - x21 +0.00005)) - np.arctan((y21-y20)/(x21-x20 +0.00005))
    angle21 = np.degrees(angle21)
    if angle21 < 0:
        angle21 = angle21 + 180.0
    if angle21>70:
        cv2.circle(img2,(x21,y21),2,255,-1)
#         print('Angle_final',angle21)
        coordd3 = (x21,y21)
        modfied_coord3.append(coordd3)
area_coord = []
area_coord2 = []
for i30 in range(len(modfied_coord3)-2):
    x30,y30 = modfied_coord3[i30]
    x31,y31 = modfied_coord3[i30+1]
    x32,y32 = modfied_coord3[i30+2]
    area = abs((x30*(y31-y32)+x31*(y32-y30)+x32*(y30-y31))/2)
#     print('Area',area)
    if area > 50:
        cv2.circle(img_out2,(x31,y31),2,255,-1)
        area_coord = (x31,y31)
        area_coord2.append(area_coord)
#         print('coord',(x31,y31))

area_coord3 = row_white + col_white + area_coord2
area_coord3 =np.array(area_coord3)

area_sorted_pts = sort(area_coord3)

area_update_pts = area_sorted_pts
def euclidean4(coords):
    x04, y04 = area_update_pts[0]
    x044, y044 = coords
    return ((x044-x04)**2 + (y044-y04)**2)**0.5
area_first_ele = []
for i80 in range(len(area_update_pts)-1):
    area_candidates = area_update_pts[1:]
#     print('SORTED\n',candidates)
    area_candidates.sort(key=euclidean4)
    area_update_pts = area_candidates  
    area_first_ele.append(area_candidates[0])

area_first_ele = np.array(area_first_ele[1:])
# print(area_first_ele)
cv2.drawContours(img_out2, [area_first_ele],  -1, (0, 0, 255), 2, cv2.LINE_AA)

mask_defect = np.zeros((rows,cols), np.uint8)
cv2.drawContours(mask_defect, [area_first_ele], 0, (255, 255, 255), -1, cv2.LINE_AA)
mask_result = cv2.bitwise_xor(mask_normal,mask_defect)

# cv2.imshow("Filled", filled)
cv2.imshow("Normal", mask_normal)
cv2.imshow("Defected", mask_defect)
cv2.imshow("Result", mask_result)
cv2.waitKey(0)
cv2.destroyAllWindows()

C:\Users\Atharva\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: overflow encountered in ubyte_scalars


In [109]:
### 2nd Checkpoint and AREA  EEEEExxxxpppppp #####
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
img = cv2.imread('./defect_2/DJI_0173_mod2.JPG')
rows = img.shape[0]
cols = img.shape[1]
rows = int((rows*40)/100)
cols = int((cols*40)/100)
img = cv2.resize(img, (cols,rows))
mask_contour = np.zeros((rows,cols), np.uint8)
img_out1 = img.copy()
img_out2 =img.copy()

hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
red = hsv[:,:,0]
rettt,thresh7 = cv2.threshold(red,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# cv2.imshow('shadows_out11', thresh7)
remove_noise = cv2.medianBlur(thresh7,3)
remove_noise2 = cv2.medianBlur(remove_noise,3)
remove_noise3 = cv2.medianBlur(remove_noise2,5)
# edges7 = cv2.Canny(remove_noise,0,255)
contours7, _ = cv2.findContours(thresh7,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
c = max(contours7, key = cv2.contourArea)
# print(c)
result = cv2.drawContours(mask_contour, [c], 0, (255, 255, 255), -1, cv2.LINE_AA)
kernel = np.ones((3,3),np.uint8)
dilation = cv2.dilate(result,kernel,iterations = 5)
filled = dilation
# filled = ~filled

result_cord= []
coord = []
for i1 in range(rows-1):
    for j1 in range(cols-1):
        pixel1 = filled[i1,j1]
        pixel2 =  filled[i1,j1+1]
        result = abs(pixel2 - pixel1)
        if result>0:
            coord = (j1+1,i1)
#             print(coord)
            result_cord.append(coord)
# print('previous',result_cord)      
row_white = []
col_white = []
whitecoordi = []
whitecoordi2 = []
for i11 in range(rows):
     for j11 in range(cols):
        white = filled[i11,j11]
        if (i11==0 and white>0) or (i11==rows-1 and white>0):
            whitecoordi = (j11,i11)
            row_white.append(whitecoordi)
        if (j11==0 and white>0) or (j11==cols-1 and white>0):
            whitecoordi2 = (j11,i11)
            col_white.append(whitecoordi2)


white_coordi = row_white + col_white + result_cord

def last(n):  
    return n[m]   
   
# function to sort the tuple    
def sort(tuples): 
    return sorted(tuples, key = last) 

m = 0
white_coordi =np.array(white_coordi)
sorted_pts = sort(white_coordi)

update_pts = sorted_pts
def euclidean(coords):
    x0, y0 = update_pts[0]
    x, y = coords
    return ((x-x0)**2 + (y-y0)**2)**0.5
first_ele = []
for i2 in range(len(update_pts)-1):
    candidates = update_pts[1:]
    candidates.sort(key=euclidean)
    update_pts = candidates
#     print('SORTED\n',candidates)
    first_ele.append(candidates[0])

# first_ele.insert(0,sorted_pts[0])
# print('\n NEW\n',first_ele)

# print('\nsorted\n',first_ele) 
values = []
central_coordi = []
#### Angle ####
for i5 in range(len(first_ele)-2):
    x5,y5 = first_ele[i5]
    x15,y15 = first_ele[i5+1]
    x25,y25 = first_ele[i5+2]
    angle2 = np.arctan((y25 - y15)/(x25 - x15 +0.00005)) - np.arctan((y15-y5)/(x15-x5 +0.00005))
    angle2 = np.degrees(angle2)
    if angle2 < 0:
        angle2 = angle2 + 180.0
    values.append(angle2)
    central_coordi.append(first_ele[i5+1])
#     print('Angle',angle2)
#     print('Coordinate',first_ele[i5+1])
  
img1 = img.copy()
img2 = img.copy()
MM=[]
angle_val = []
for i6 in range(len(values)-1):
    val1 = values[i6]
    val2 = values[i6+1]
    val3 = abs(val2-val1)
    X1,Y1 = central_coordi[i6]
    X2,Y2 = central_coordi[i6+1]
    val4 = abs(X2-X1)
    M = (val3+0.00005)/(val4+0.00005)
    MM.append(M)
    angle_val.append(val3)
#     print('Measure',M)
#     print('Coordinate',central_coordi[i6])

medi1 = np.mean(MM)
# print('Mean',medi1)
coordd = []
modfies_coord = []
for i9 in range(len(MM)):
    if MM[i9]>medi1*4.5:
            X1,Y1 = central_coordi[i9]
            coordd = (X1,Y1)
            modfies_coord.append(coordd)
#             cv2.circle(img1,(X1,Y1),2,255,-1)
coordd2 = []
modfied_coord2 = []
for i10 in range(len(modfies_coord)-1):
    x10,y10 = modfies_coord[i10]
    x100,y100 = modfies_coord[i10+1]

    distance = ((x100-x10)**2 + (y100-y10)**2)**0.5
    if distance > 6:
#         print('Distance',distance)
#         cv2.circle(img1,(x10,y10 ),2,255,-1)


        coordd2 = (x10,y10)
        modfied_coord2.append(coordd2)
coordd3 = []
modfied_coord3 = []
for i20 in range(len(modfied_coord2)-2):
    x20,y20 = modfied_coord2[i20]
    x21,y21 = modfied_coord2[i20+1]
    x22,y22 = modfied_coord2[i20+2]
    angle21 = np.arctan((y22 - y21)/(x22 - x21 +0.00005)) - np.arctan((y21-y20)/(x21-x20 +0.00005))
    angle21 = np.degrees(angle21)
    if angle21 < 0:
        angle21 = angle21 + 180.0
    if angle21>70:
        cv2.circle(img2,(x21,y21),2,255,-1)
#         print('Angle_final',angle21)
        coordd3 = (x21,y21)
        modfied_coord3.append(coordd3)
img3 =img.copy()     
area_coord = []
area_coord2 = []
for i30 in range(len(modfied_coord3)-2):
    x30,y30 = modfied_coord3[i30]
    x31,y31 = modfied_coord3[i30+1]
    x32,y32 = modfied_coord3[i30+2]
    area = abs((x30*(y31-y32)+x31*(y32-y30)+x32*(y30-y31))/2)
#     print('Area',area)
    if area > 50:
        cv2.circle(img2,(x31,y31),2,255,-1)
        area_coord = (x31,y31)
        area_coord2.append(area_coord)
#        
print('Area points',area_coord2)
print('first coordi',area_coord2[0])
print('last coordi',area_coord2[-1])
boundry = row_white + col_white
print('boarder coordi',boundry)
# area_coord3 =np.array(area_coord3)



# area_update_pts = area_sorted_pts
def euclidean4(coords):
    x04, y04 = area_coord2[0]
    x044, y044 = coords
    return ((x044-x04)**2 + (y044-y04)**2)**0.5
start_ele = []
start_candidates = boundry[1:]
start_candidates.sort(key=euclidean4)  
start_ele.append(start_candidates)

def euclidean5(coords):
    x05, y05 = area_coord2[-1]
    x055, y055 = coords
    return ((x055-x05)**2 + (y055-y05)**2)**0.5
end_ele = []
end_candidates = boundry[1:]
end_candidates.sort(key=euclidean5)  
end_ele.append(end_candidates)
   
# area_first_ele = np.array(area_first_ele[1:])
startpart = start_ele[0]
endpart = end_ele[0]
def Reverse(lst): 
    return [ele for ele in reversed(lst)]
start_coordi=Reverse(startpart)
start_coordi = start_coordi[-30:]
end_coordi= endpart[:-40]
print('\nstart\n',start_coordi)
print('\nend\n',endpart)
print('\nend2\n',end_coordi)

final_contour = start_coordi + area_coord2 + end_coordi
print('\nfinal contour points\n',final_contour)
final_contour = np.array(final_contour)
cv2.drawContours(img1, [final_contour],  -1, (0, 0, 255), 4, cv2.LINE_AA)

cv2.imshow("Output:middle", img1)

# cv2.imshow("masked", mask1)
# cv2.imshow("masked_G-channel", mask2)
cv2.waitKey(0)
cv2.destroyAllWindows()

C:\Users\Atharva\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: overflow encountered in ubyte_scalars


Area points [(60, 22), (61, 28), (97, 14), (130, 42), (159, 37), (180, 27), (234, 105), (267, 89), (263, 147), (285, 156), (232, 189), (226, 195), (209, 174), (161, 197), (177, 210), (205, 243), (210, 266)]
first coordi (60, 22)
last coordi (210, 266)
boarder coordi [(4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (15, 0), (16, 0), (17, 0), (18, 0), (19, 0), (20, 0), (21, 0), (22, 0), (23, 0), (24, 0), (25, 0), (26, 0), (27, 0), (28, 0), (29, 0), (30, 0), (0, 275), (1, 275), (2, 275), (3, 275), (4, 275), (5, 275), (6, 275), (7, 275), (8, 275), (9, 275), (10, 275), (11, 275), (12, 275), (13, 275), (14, 275), (15, 275), (16, 275), (17, 275), (18, 275), (19, 275), (20, 275), (21, 275), (22, 275), (23, 275), (24, 275), (25, 275), (26, 275), (27, 275), (28, 275), (29, 275), (30, 275), (31, 275), (32, 275), (33, 275), (34, 275), (35, 275), (36, 275), (37, 275), (38, 275), (39, 275), (40, 275), (41, 275), (42, 275), (43, 275), (44, 275), (45, 275),

In [13]:
### 2nd Checkpoint and AREA #####
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
img = cv2.imread('./defect_2/DJI_0173_mod3.JPG')
rows = img.shape[0]
cols = img.shape[1]
rows = int((rows*40)/100)
cols = int((cols*40)/100)
img = cv2.resize(img, (cols,rows))
mask_contour = np.zeros((rows,cols), np.uint8)
img_out1 = img.copy()
img_out2 =img.copy()

hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
red = hsv[:,:,0]
rettt,thresh7 = cv2.threshold(red,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# cv2.imshow('shadows_out11', thresh7)
remove_noise = cv2.medianBlur(thresh7,3)
remove_noise2 = cv2.medianBlur(remove_noise,3)
remove_noise3 = cv2.medianBlur(remove_noise2,5)
# edges7 = cv2.Canny(remove_noise,0,255)
contours7, _ = cv2.findContours(thresh7,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
c = max(contours7, key = cv2.contourArea)
# print(c)
result = cv2.drawContours(mask_contour, [c], 0, (255, 255, 255), -1, cv2.LINE_AA)
kernel = np.ones((3,3),np.uint8)
dilation = cv2.dilate(result,kernel,iterations = 5)
filled = dilation
# filled = ~filled

result_cord= []
coord = []
for i1 in range(rows-1):
    for j1 in range(cols-1):
        pixel1 = filled[i1,j1]
        pixel2 =  filled[i1,j1+1]
        result = abs(pixel2 - pixel1)
        if result>0:
            coord = (j1+1,i1)
#             print(coord)
            result_cord.append(coord)
# print('previous',result_cord)      
row_white = []
col_white = []
whitecoordi = []
whitecoordi2 = []
for i11 in range(rows):
     for j11 in range(cols):
        white = filled[i11,j11]
        if (i11==0 and white>0) or (i11==rows-1 and white>0):
            whitecoordi = (j11,i11)
            row_white.append(whitecoordi)
        if (j11==0 and white>0) or (j11==cols-1 and white>0):
            whitecoordi2 = (j11,i11)
            col_white.append(whitecoordi2)


white_coordi = row_white + col_white + result_cord

def last(n):  
    return n[m]   
   
# function to sort the tuple    
def sort(tuples): 
    return sorted(tuples, key = last) 

m = 0
white_coordi =np.array(white_coordi)
sorted_pts = sort(white_coordi)

update_pts = sorted_pts
def euclidean(coords):
    x0, y0 = update_pts[0]
    x, y = coords
    return ((x-x0)**2 + (y-y0)**2)**0.5
first_ele = []
for i2 in range(len(update_pts)-1):
    candidates = update_pts[1:]
    candidates.sort(key=euclidean)
    update_pts = candidates
#     print('SORTED\n',candidates)
    first_ele.append(candidates[0])

# first_ele.insert(0,sorted_pts[0])
# print('\n NEW\n',first_ele)

# print('\nsorted\n',first_ele) 
values = []
central_coordi = []
#### Angle ####
for i5 in range(len(first_ele)-2):
    x5,y5 = first_ele[i5]
    x15,y15 = first_ele[i5+1]
    x25,y25 = first_ele[i5+2]
    angle2 = np.arctan((y25 - y15)/(x25 - x15 +0.00005)) - np.arctan((y15-y5)/(x15-x5 +0.00005))
    angle2 = np.degrees(angle2)
    if angle2 < 0:
        angle2 = angle2 + 180.0
    values.append(angle2)
    central_coordi.append(first_ele[i5+1])
#     print('Angle',angle2)
#     print('Coordinate',first_ele[i5+1])
  
img1 = img.copy()
img2 = img.copy()
MM=[]
angle_val = []
for i6 in range(len(values)-1):
    val1 = values[i6]
    val2 = values[i6+1]
    val3 = abs(val2-val1)
    X1,Y1 = central_coordi[i6]
    X2,Y2 = central_coordi[i6+1]
    val4 = abs(X2-X1)
    M = (val3+0.00005)/(val4+0.00005)
    MM.append(M)
    angle_val.append(val3)
#     print('Measure',M)
#     print('Coordinate',central_coordi[i6])

medi1 = np.mean(MM)
# print('Mean',medi1)
coordd = []
modfies_coord = []
for i9 in range(len(MM)):
    if MM[i9]>medi1*4.5:
            X1,Y1 = central_coordi[i9]
            coordd = (X1,Y1)
            modfies_coord.append(coordd)
#             cv2.circle(img1,(X1,Y1),2,255,-1)
coordd2 = []
modfied_coord2 = []
for i10 in range(len(modfies_coord)-1):
    x10,y10 = modfies_coord[i10]
    x100,y100 = modfies_coord[i10+1]

    distance = ((x100-x10)**2 + (y100-y10)**2)**0.5
    if distance > 6:
#         print('Distance',distance)
#         cv2.circle(img1,(x10,y10 ),2,255,-1)


        coordd2 = (x10,y10)
        modfied_coord2.append(coordd2)
coordd3 = []
modfied_coord3 = []
for i20 in range(len(modfied_coord2)-2):
    x20,y20 = modfied_coord2[i20]
    x21,y21 = modfied_coord2[i20+1]
    x22,y22 = modfied_coord2[i20+2]
    angle21 = np.arctan((y22 - y21)/(x22 - x21 +0.00005)) - np.arctan((y21-y20)/(x21-x20 +0.00005))
    angle21 = np.degrees(angle21)
    if angle21 < 0:
        angle21 = angle21 + 180.0
    if angle21>70:
        cv2.circle(img2,(x21,y21),2,255,-1)
#         print('Angle_final',angle21)
        coordd3 = (x21,y21)
        modfied_coord3.append(coordd3)
img3 =img.copy()     
area_coord = []
area_coord2 = []
for i30 in range(len(modfied_coord3)-2):
    x30,y30 = modfied_coord3[i30]
    x31,y31 = modfied_coord3[i30+1]
    x32,y32 = modfied_coord3[i30+2]
    area = abs((x30*(y31-y32)+x31*(y32-y30)+x32*(y30-y31))/2)
#     print('Area',area)
    if area > 50:
        cv2.circle(img2,(x31,y31),2,255,-1)
        area_coord = (x31,y31)
        area_coord2.append(area_coord)
#         print('coord',(x31,y31))

area_coord3 = row_white + col_white + area_coord2
area_coord3 =np.array(area_coord3)

area_sorted_pts = sort(area_coord3)

area_update_pts = area_sorted_pts
def euclidean4(coords):
    x04, y04 = area_update_pts[0]
    x044, y044 = coords
    return ((x044-x04)**2 + (y044-y04)**2)**0.5
area_first_ele = []
for i80 in range(len(area_update_pts)-1):
    area_candidates = area_update_pts[1:]
#     print('SORTED\n',candidates)
    area_candidates.sort(key=euclidean4)
    area_update_pts = area_candidates  
    area_first_ele.append(area_candidates[0])

area_first_ele = np.array(area_first_ele[1:])

print(area_first_ele)



cv2.drawContours(img1, [area_first_ele],  -1, (0, 0, 255), 4, cv2.LINE_AA)
####### Second order differen ####            
# second_ord11 =[]
# print(len(MM))
# for i7 in range(len(MM)-1):
#     second_ord = abs(MM[i7+1]-MM[i7])
#     X11,Y11 = central_coordi[i7]
#     X22,Y22 = central_coordi[i7+1]
#     val5 = abs(X22-X11)
#     second_ord1 = second_ord/(val5+0.00005)
#     second_ord11.append(second_ord1)
# #     print('Second Ord:',second_ord1)
# #     
# medi = np.mean(second_ord11)
# print('Mean',medi)
# for i8 in range(len(second_ord11)):
#     if second_ord11[i8]>medi*4.5:
#             XX1,YY1 = central_coordi[i8]
#             cv2.circle(img2,(XX1,YY1),2,255,-1)
##########################
# cv2.namedWindow("Output:middle",cv2.WINDOW_NORMAL)
cv2.imshow("Output:middle", img1)

# cv2.imshow("masked", mask1)
# cv2.imshow("masked_G-channel", mask2)
cv2.waitKey(0)
cv2.destroyAllWindows()

C:\Users\Atharva\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: overflow encountered in ubyte_scalars


[[  0 281]
 [  0 280]
 [  0 279]
 [  0 278]
 [  0 277]
 [  0 276]
 [  0 275]
 [  0 274]
 [  0 273]
 [  0 272]
 [  0 271]
 [  0 270]
 [  0 269]
 [  0 268]
 [  0 267]
 [  0 266]
 [  0 265]
 [  0 264]
 [  0 263]
 [  0 262]
 [  0 261]
 [  0 260]
 [  0 259]
 [  0 258]
 [  0 257]
 [  0 256]
 [  0 255]
 [  0 254]
 [  0 253]
 [  0 252]
 [  0 251]
 [  0 250]
 [  0 249]
 [  0 248]
 [  0 247]
 [  0 246]
 [  0 245]
 [  0 244]
 [  0 243]
 [  0 242]
 [  0 241]
 [  0 240]
 [  0 239]
 [  0 238]
 [  0 237]
 [  0 236]
 [  0 235]
 [  0 234]
 [  0 233]
 [  0 232]
 [  7 224]
 [ 19 203]
 [ 19 282]
 [ 18 282]
 [ 17 282]
 [ 16 282]
 [ 15 282]
 [ 14 282]
 [ 13 282]
 [ 12 282]
 [ 11 282]
 [ 10 282]
 [  9 282]
 [  8 282]
 [  7 282]
 [  6 282]
 [  5 282]
 [  4 282]
 [  3 282]
 [  2 282]
 [  1 282]
 [ 20 282]
 [ 21 282]
 [ 22 282]
 [ 23 282]
 [ 24 282]
 [ 25 282]
 [ 26 282]
 [ 27 282]
 [ 28 282]
 [ 29 282]
 [ 30 282]
 [ 31 282]
 [ 32 282]
 [ 33 282]
 [ 34 282]
 [ 35 282]
 [ 36 282]
 [ 37 282]
 [ 38 282]
 [ 39 282]

In [74]:
 ### convex hull outer contour ### 
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
img = cv2.imread('./defected/leaf22_modified.jpg')
rows = img.shape[0]
cols = img.shape[1]
# rows = int((rows*40)/100)
# cols = int((cols*40)/100)
# img = cv2.resize(img, (cols,rows)) 

g = img[:,:,1]
kernel = np.ones((2,2),np.uint8)
eroded2 = cv2.erode(g,kernel,iterations = 3)
ret4,thresh4 = cv2.threshold(eroded2,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

edges = cv2.Canny(thresh4,0,255)
ppp,contours,hier = cv2.findContours(edges,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
filled = cv2.drawContours(thresh4,contours,-1,(255,255,255),15)

# filled = ~filled

result_cord= []
coord = []
for i1 in range(rows-1):
    for j1 in range(cols-1):
        pixel1 = filled[i1,j1]
        pixel2 =  filled[i1,j1+1]
        result = abs(pixel2 - pixel1)
        if result>0:
            coord = (j1+1,i1)
#             print(coord)
            result_cord.append(coord)
# print('previous',result_cord)      
row_white = []
col_white = []
whitecoordi = []
whitecoordi2 = []
for i11 in range(rows):
     for j11 in range(cols):
        white = filled[i11,j11]
        if (i11==0 and white>0) or (i11==rows-1 and white>0):
            whitecoordi = (j11,i11)
            row_white.append(whitecoordi)
        if (j11==0 and white>0) or (j11==cols-1 and white>0):
            whitecoordi2 = (j11,i11)
            col_white.append(whitecoordi2)


white_coordi = row_white + col_white + result_cord
white_coordi = np.array(white_coordi)

hull = cv2.convexHull(white_coordi)
hull_coord1 = []
hull_coord2 = []
for i4 in range(len(hull)):
    hull_coord = hull[i4]
    x4, y4 = hull_coord[0]
    hull_coord1 = (x4, y4)
    hull_coord2.append(hull_coord1)
hull_new =  row_white + col_white + hull_coord2 
hull_new = np.array(hull_new)

def last(n):  
    return n[m]   
def sort(tuples): 
    return sorted(tuples, key = last) 
m = 0
hull_sorted_pts = sort(hull_new)

hull_update_pts = hull_sorted_pts
def euclidean(coords):
    x0, y0 = hull_update_pts[0]
    x, y = coords
    return ((x-x0)**2 + (y-y0)**2)**0.5
hull_first_ele = []
for i2 in range(len(hull_update_pts)-1):
    candidates = hull_update_pts[1:]
#     print('SORTED\n',candidates)
    candidates.sort(key=euclidean)
    hull_update_pts = candidates  
    hull_first_ele.append(candidates[0])

hull_first_ele = np.array(hull_first_ele[1:])

def euclidean2(coords):
    x60, y60 = hull_first_ele[-1]
    x6, y6 = coords
    return ((x6-x60)**2 + (y6-y60)**2)**0.5
connect = sort(hull_first_ele[:100])
for i66 in range(len(connect)):
    connect.sort(key = euclidean2)
f0,f1 = connect[0] 
f2,f3 = hull_first_ele[-1]
# for i55 in range(len(hull_first_ele)-1):
#     x50,y50 = hull_first_ele[i55]
#     x51,y51 = hull_first_ele[i55+1]
#     lineThickness = 2
#     cv2.line(img, (x50,y50), (x51,y51), (0,0,255), lineThickness)
#     cv2.line(img, (f0,f1), (f2,f3), (0,0,255), lineThickness)
hull_first_ele2 =[]
hull_first_ele3 =[]
for i70 in range(len(hull_first_ele)):
    f2,f3 = hull_first_ele[i70]
    if (f2,f3)!=(f0,f1):
        hull_first_ele2 = (f2,f3)
        hull_first_ele3.append(hull_first_ele2)

# hull_first_ele3 = hull_first_ele3.insert(0,connect[0])
hull_first_ele3 = [(f0,f1)]+hull_first_ele3
hull_first_ele3 = np.array(hull_first_ele3)
print(hull_first_ele3)
cv2.drawContours(img, [hull_first_ele3],  -1, (0, 0, 255), 2, cv2.LINE_AA)

cv2.imshow("OUTPUT2", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

/home/atharva/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: overflow encountered in ubyte_scalars


[[  0 347]
 [  0 446]
 [  0 445]
 ...
 [180   0]
 [180   0]
 [ 81 117]]


In [59]:
### Centroid for stem ####

import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
img = cv2.imread('leaf20.jpg')
rows = img.shape[0]
cols = img.shape[1]
rows = int((rows*40)/100)
cols = int((cols*40)/100)
img = cv2.resize(img, (cols,rows)) 

g = img[:,:,1]
kernel = np.ones((2,2),np.uint8)
eroded2 = cv2.erode(g,kernel,iterations = 3)
ret4,thresh4 = cv2.threshold(eroded2,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

edges = cv2.Canny(thresh4,0,255)
ppp,contours,hier = cv2.findContours(edges,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
filled = cv2.drawContours(thresh4,contours,-1,(255,255,255),15)
(winW, winH) = (cols, 35)
stepSize = 10
blob = []
centroid = []
for height in range(0, rows, stepSize):
    crop = filled[height:height + winH, 0:0 + winW]
    M = cv2.moments(crop)
    cv2.imshow("OUTPUT1", crop)
    # calculate x,y coordinate of center
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    blob = (cX, height+cY)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    centroid.append(blob)
centers = np.int0(centroid)
for center in centers:
    x,y = center.ravel()
    cv2.circle(img,(x,y),3,(0,0,255),-1)
cv2.imshow("OUTPUT2", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
### 1st Checkpoint #####
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
img = cv2.imread('./defect_2/DJI_0173_mod3.JPG')
rows = img.shape[0]
cols = img.shape[1]
rows = int((rows*40)/100)
cols = int((cols*40)/100)
img = cv2.resize(img, (cols,rows))
mask_contour = np.zeros((rows,cols), np.uint8)
img_out1 = img.copy()
img_out2 =img.copy()

hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
red = hsv[:,:,0]
rettt,thresh7 = cv2.threshold(red,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

cv2.imshow('shadows_out11', thresh7)
remove_noise = cv2.medianBlur(thresh7,3)
remove_noise2 = cv2.medianBlur(remove_noise,3)
remove_noise3 = cv2.medianBlur(remove_noise2,5)
# edges7 = cv2.Canny(remove_noise,0,255)
contours7, _ = cv2.findContours(thresh7,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
c = max(contours7, key = cv2.contourArea)
# print(c)
result = cv2.drawContours(mask_contour, [c], 0, (255, 255, 255), -1, cv2.LINE_AA)
kernel = np.ones((3,3),np.uint8)
dilation = cv2.dilate(result,kernel,iterations = 5)
filled = dilation

# filled = ~filled

result_cord= []
coord = []
for i1 in range(rows-1):
    for j1 in range(cols-1):
        pixel1 = filled[i1,j1]
        pixel2 =  filled[i1,j1+1]
        result = abs(pixel2 - pixel1)
        if result>0:
            coord = (j1+1,i1)
#             print(coord)
            result_cord.append(coord)
# print('previous',result_cord)      
row_white = []
col_white = []
whitecoordi = []
whitecoordi2 = []
for i11 in range(rows):
     for j11 in range(cols):
        white = filled[i11,j11]
        if (i11==0 and white>0) or (i11==rows-1 and white>0):
            whitecoordi = (j11,i11)
            row_white.append(whitecoordi)
        if (j11==0 and white>0) or (j11==cols-1 and white>0):
            whitecoordi2 = (j11,i11)
            col_white.append(whitecoordi2)

# print('ROW\n',row_white)
# print('COL\n',col_white)
white_coordi = row_white + col_white + result_cord
# corners = np.int0(white_coordi)
# for corner in corners:
#     x,y = corner.ravel()
#     cv2.circle(img,(x,y),3,255,-1)
    
# print('COL\n',white_coordi)
def last(n):  
    return n[m]   
   
# function to sort the tuple    
def sort(tuples): 
    return sorted(tuples, key = last) 

m = 0
sorted_pts = sort(white_coordi)

update_pts = sorted_pts
def euclidean(coords):
    x0, y0 = update_pts[0]
    x, y = coords
    return ((x-x0)**2 + (y-y0)**2)**0.5
first_ele = []
for i2 in range(len(update_pts)-1):
    candidates = update_pts[1:]
    candidates.sort(key=euclidean)
    update_pts = candidates
#     print('SORTED\n',candidates)
    first_ele.append(candidates[0])

first_ele.insert(0,sorted_pts[0])
# print('\n NEW\n',first_ele)

print('\nsorted\n',len(first_ele)) 
img111 = img.copy()
img222 = img.copy()

for i2 in range(len(first_ele)-1):
    x0,y0 = first_ele[i2]
    x1,y1 = first_ele[i2+1]
    lineThickness = 2
    cv2.line(img111, (x0, y0), (x1,y1), (0,0,255), lineThickness)
    distance = ((x1-x0)**2 + (y1-y0)**2)**0.5
    if distance <= 120:
        lineThickness = 2
        cv2.line(img222, (x0, y0), (x1,y1), (0,0,255), lineThickness)
## Mask ##
mask1 = cv2.bitwise_or(img, img, mask=filled)

first_ele = np.array(first_ele)
mask2 = mask1[:,:,1]

pixels = np.float32(mask2.reshape(-1, 1))
print(pixels.shape)
n_colors = 5
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
flags = cv2.KMEANS_RANDOM_CENTERS

_, labels, palette = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)
_, counts = np.unique(labels, return_counts=True)

palette = np.uint8(palette)
res = palette[labels.flatten()]
res2 = res.reshape((mask2.shape))

green_pix = 0
other_pix = 0
for j14 in range(rows):
    for j24 in range(cols):
        pixel_val = res2[j14,j24]
        if pixel_val>= 1 and pixel_val<= 200:
            green_pix +=1
        elif pixel_val>200:
            other_pix +=1
print('Green:',green_pix)
print('Other:',other_pix)
        
# dominant = palette[np.argmax(counts)]
print('Dominant',counts)
total = green_pix+other_pix
percentage = (green_pix*100)/total
print(percentage)


# cv2.imwrite("./results/Green_channel.jpg", g)
# cv2.imwrite("./results/Noise_removal:- erosion,otsu-thresh,Canny_edge_detection.jpg", edges)
# cv2.imwrite("./results/contour_filling.jpg", filled)
# cv2.imwrite("./results/Output:boarder.jpg", img222)
# cv2.imshow("Green_channel", g)
# cv2.imshow("Noise_removal:- erosion,otsu-thresh,Canny_edge_detection", edges)
# cv2.imshow("results/contour_filling", filled)
cv2.imshow("Output:boarder", filled)
cv2.imshow("masked", mask1)
cv2.imshow("masked_G-channel", mask2)
cv2.waitKey(0)
cv2.destroyAllWindows()

C:\Users\Atharva\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: overflow encountered in ubyte_scalars



sorted
 1607
(113766, 1)
Green: 40959
Other: 8363
Dominant [14844  8363 15460 64444 10655]
83.04407769352419


In [27]:
### GOOD Feature & Contour #####
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
img = cv2.imread('./defected/leaf22_modified.jpg')
rows = img.shape[0]
# rows = int((rows*40)/100)
cols = img.shape[1]
# cols = int((cols*40)/100)
# img = cv2.resize(img, (cols,rows))

g = img[:,:,1]
kernel = np.ones((2,2),np.uint8)
eroded2 = cv2.erode(g,kernel,iterations = 3)
ret4,thresh4 = cv2.threshold(eroded2,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

edges = cv2.Canny(thresh4,0,255)
ppp,contours,hier = cv2.findContours(edges,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
filled = cv2.drawContours(thresh4,contours,-1,(255,255,255),6)

filled = ~filled

corners = cv2.goodFeaturesToTrack(filled, 1000, 0.005, 0.5)
corners = np.int0(corners)

pts = []
key_pts = []
for corner in corners:
    x,y = corner.ravel()
    cv2.circle(img,(x,y),3,255,-1)
    pts = (x,y)
    key_pts.append(pts)


# Sorting
def last(n): 
    return n[m]   
   
# function to sort the tuple    
def sort(tuples): 
    return sorted(tuples, key = last) 

m = 0
sorted_pts = sort(key_pts)
# print('Sorted:',sorted_pts) 

X_cordinates = []
Y_cordinates = []
for i1 in range(len(sorted_pts)):
    x_cor,y_cor = sorted_pts[i1]
    X_cordinates.append(x_cor)
    Y_cordinates.append(y_cor)
# print('X_coordinates:',X_cordinates[0])
# print('Y_coordinates:',Y_cordinates[0])

update_pts = sorted_pts

def euclidean(coords):
    x0, y0 = update_pts[0]
    x, y = coords
    return ((x-x0)**2 + (y-y0)**2)**0.5
first_ele = []
for i2 in range(len(update_pts)-1):
    candidates = update_pts[1:]
    candidates.sort(key=euclidean)
    update_pts = candidates
#     print('SORTED\n',candidates)
    first_ele.append(candidates[0])

first_ele.insert(0,sorted_pts[0])
print('\n NEW\n',first_ele)

img1 = img.copy()
img2 = img.copy()
for i3 in range(len(first_ele)-1):
    x00,y00 = first_ele[i3]
    x11,y11 = first_ele[i3+1]
    lineThickness = 2
    cv2.line(img1, (x00, y00), (x11,y11), (0,0,255), lineThickness)
    
    distance = ((x11-x00)**2 + (y11-y00)**2)**0.5
    if distance <= 120:
        lineThickness = 2
        cv2.line(img2, (x00, y00), (x11,y11), (0,0,255), lineThickness)
#         cv2.line(mask, (x00, y00), (x11,y11), (0,0,255), lineThickness)

## Mask ##
first_ele_array = np.array(first_ele)
mask = np.zeros((rows,cols), np.uint8)
cv2.drawContours(mask, [first_ele_array], 0, (255, 255, 255), -1, cv2.LINE_AA)
# print(mask.shape)
# print(img1.shape)
mask1 = cv2.bitwise_and(img, img, mask=mask)

mask1 = mask1[:,:,1]
print(mask1.shape)
# average = mask1.mean(axis=0).mean(axis=0)
pixels = np.float32(mask1.reshape(-1, 1))
print(pixels.shape)
n_colors = 3
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
flags = cv2.KMEANS_RANDOM_CENTERS

_, labels, palette = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)
_, counts = np.unique(labels, return_counts=True)

palette = np.uint8(palette)
res = palette[labels.flatten()]
res2 = res.reshape((mask1.shape))

green_pix = 0
other_pix = 0
for j1 in range(rows):
    for j2 in range(cols):
        pixel_val = res2[j1,j2]
        if pixel_val>= 1 and pixel_val<= 150:
            green_pix +=1
        elif pixel_val>150:
            other_pix +=1
print('Green:',green_pix)
print('Other:',other_pix)
        
# dominant = palette[np.argmax(counts)]
print('Dominant',counts)
total = green_pix+other_pix
percentage = (green_pix*100)/total
print(percentage)
# cv2.namedWindow("output", cv2.WINDOW_NORMAL)   
imS = cv2.resize(img2, (400, 600))                    # Resize image
# cv2.imshow("output", imS)    
cv2.imshow("output1", filled) 
cv2.imshow("output2", imS) 
cv2.imshow("output3", res2)
cv2.waitKey(0)
cv2.destroyAllWindows()




 NEW
 [(5, 334), (6, 334), (8, 336), (9, 335), (11, 341), (18, 337), (22, 338), (26, 338), (29, 337), (36, 335), (33, 330), (34, 329), (32, 327), (32, 326), (33, 324), (34, 321), (34, 319), (34, 316), (36, 313), (41, 311), (42, 305), (41, 303), (37, 300), (38, 299), (36, 296), (37, 293), (38, 291), (39, 288), (38, 285), (36, 283), (36, 282), (38, 279), (40, 277), (42, 273), (42, 272), (43, 271), (43, 270), (45, 267), (50, 264), (51, 257), (51, 256), (51, 243), (48, 242), (46, 240), (45, 241), (44, 238), (44, 237), (46, 234), (50, 231), (53, 226), (55, 224), (58, 222), (57, 221), (60, 220), (62, 219), (63, 219), (66, 220), (68, 221), (70, 222), (71, 222), (73, 223), (76, 224), (79, 225), (81, 226), (82, 226), (85, 226), (87, 224), (90, 222), (98, 221), (101, 220), (106, 219), (109, 219), (111, 216), (111, 215), (112, 214), (112, 213), (113, 211), (113, 205), (113, 204), (112, 199), (112, 198), (111, 195), (111, 194), (110, 188), (110, 187), (109, 183), (106, 183), (105, 183), (101, 183